In [34]:
# Khai báo sử dụng các thư viện. Thực hiện chạy ngay trên CPU, do đó, chỉ thực hiện mô phỏng, nếu ổn định sẽ thực hiện chạy trên kaggle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

import regex as re
import os
import hashlib
from datetime import datetime
import hashlib
import copy

import glob # Thư viện mở rộng tên đường dẫn theo kiểu Unix

from tabulate import tabulate # Thư viện sử dụng hiển thị dữ liệu dạng bảng cho DataFrame

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [35]:
# Chỉnh sửa class Logcluster:
from Root_Drain import LogParser, Node, Logcluster            # Source code cũ của Drain

class Logcluster3(Logcluster):
    """
    Class được chỉnh sửa lại, bao gồm một số thông số như sau:
    
    Thuộc tính:
    ----------
        - `logTemplate`: Template đặc trưng đại diện cho nhóm log đó, ["The1", "the2", ...]
        - `logIDL`     : Danh sách các ID message log thuộc nhóm log trên, [1,2,3,4,5, ...] 
        - `levelL`     : Danh sách các Level (INFO, WARN, FATAL, ERROR) mà nhóm log có thể biểu diễn, ["WARN", "INFO", ...]
        - `idTemplate` : Mã định danh của template đó, có thể bị thay đổi do template.
        - `idLevelTemp`: Xác định mã định danh theo dạng {"INFO": "L1", ...} 
    """
    def __init__(self, logTemplate="", levelL=None, logIDL=None):
        self.logTemplate = logTemplate
        if logIDL is None:
            logIDL = []
        self.logIDL = logIDL
        self.levelL = list(levelL)
        self.idTemplate = None
        self.idLevelTemp = {"TEMP":None}
        
    def addIDLevel(self, logLevel):
        self.levelL.append(logLevel)



In [36]:
# Thực hiện chỉnh sửa lại 2 phương thức outputResult() và parser() của Drain:
# Kế thừa Drain bằng cách tạo ra class Drain2:
class LogParser2(LogParser):
    """
    Kế thừa từ LogParser được sử dụng để thực hiện sửa đổi lại sao cho phù hợp.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)       # Đọc thêm: https://viblo.asia/p/python-args-va-kwargs-gDVK2pdnlLj
        self.rootNode = Node()
        self.list_logClust = []
        self.list_eventsID = {}
        self.df_TemplateL = None
        self.list_counts = {"TEMP":0}
    
    def convert_unixtime(self, row):
        """
        Phương thức trích xuất thời gian theo dạng chuẩn UNIX
        """
        date_str = row["Date"]
        time_str = row["Time"]
        datetime_str = date_str[:2] + "-" + date_str[2:4] + "-" + date_str[4:] + " " + time_str[:2] + ":" + time_str[2:4] + ":" + time_str[4:]
        unixtime = datetime.strptime(datetime_str, "%m-%y-%d %H:%M:%S").timestamp()
        return int(unixtime)
    
    # def extract_block_id(self, parameter_list):
    #     """
    #     Phương thức trích xuất blockID cho từng thông điệp log
    #     """
    #     block_ids = []
    #     pattern = re.compile(r'\bblk_-?\d+\b')  # Biểu thức chính quy để tìm blockID
    #     for param in parameter_list:
    #         matches = pattern.findall(param)
    #         if matches:  # Kiểm tra nếu tìm thấy blockID
    #             block_ids.extend(matches)
    #     return block_ids

    def extract_block_id(self, parameter_list):
        """
        Phương thức trích xuất blockID cho từng thông điệp log

        Trả về:
            - Chuỗi blockID đầu tiên được tìm thấy hoặc `NaN` nếu không tìm thấy.
        """
        pattern = re.compile(r'\bblk_-?\d+\b')  # Biểu thức chính quy để tìm blockID
        for param in parameter_list:
            matches = pattern.findall(param)
            if matches:  # Kiểm tra nếu tìm thấy blockID
                return matches[0]
        return np.nan

    # def extractByBlockID(self):
    #     """
    #         Trích xuất ra quy trình template sử dụng theo BlockID
    #     """
    #     unique_values = self.df_log['BlockID'].unique()
    #     unique_values = [x for x in unique_values if pd.notna(x)]

    #     res = {}

    #     for value in unique_values:
    #         filtered_df = self.df_log[self.df_log['BlockID'] == value].sort_values(by='UnixTime')
    #         res[value] = filtered_df['IDLevel'].values.tolist()
        
    #     return res
    
    def extractByBlockID(self):
        """
            Trích xuất ra quy trình template sử dụng theo BlockID
        """
        unique_values = self.df_log['BlockID'].unique()
        unique_values = [x for x in unique_values if pd.notna(x)]
        
        res = {block_id: [] for block_id in unique_values}
        self.df_log = self.df_log.sort_values(by="UnixTime")
        
        for idx, row in self.df_log.iterrows(): 
            level = row["IDLevel"]
            res[row["BlockID"]].append(level)
        
        return res
    
    def outputResult(self, logClustL):
        """
        Thực hiện in thông tin kết quả ra ngoài sau khi hoàn thành quá trình phân tích
        
        Tham số:
        --------
            - `logClustL`   : Danh sách các nhóm log đã thu thập được trong quá trình phân tách
        """
        # ----------- Khai báo các biến sử dụng ----------#
        log_templates = [0] * self.df_log.shape[0]
        log_templateids = [0] * self.df_log.shape[0]
        list_events = []                            # Mảng list_events lưu giữ các giá trị template
        dictionary_events = self.list_eventsID      # Dictionary lưu giữ các đối tượng idTemplate và các thông số của nó: "idTemplate": {"TEMP":"L5", "INFO":"I2", ...}
    
        # --------------Thực hiện phân tích--------------#
        # * Duyệt qua từng đối tượng Logcluster để thêm các giá trị vào từng biến trên
        for logClust in logClustL:
            template_str = " ".join(logClust.logTemplate)
            occurrence = len(logClust.logIDL)
            template_id = hashlib.md5(template_str.encode("utf-8")).hexdigest()[0:8]
            logClust.idTemplate = template_id   # không cần cũng được
            for logID in logClust.logIDL:
                logID -= 1
                log_templates[logID] = template_str
                log_templateids[logID] = template_id
            logClust.logIDL=[]
            list_events.append([template_id, template_str, occurrence, logClust.levelL, logClust.idLevelTemp])
            if template_id not in dictionary_events:
                dictionary_events[template_id] = logClust.idLevelTemp

        # DataFrame df_event lưu giữ các thông tin của Template
        df_event = pd.DataFrame(
            list_events, columns=["EventId", "EventTemplate", "Occurrences", "logLevel", "IDlogLevel"]
        )
        # varL1.df_all_events = df_event
        
        # DataFrame df_log lưu giữ các thông tin của từng dòng thông điệp log
        self.df_log["EventId"] = log_templateids
        self.df_log["EventTemplate"] = log_templates
        
        # Chuyển đổi thời gian theo dạng UnixTime:
        # Tạo thêm cột "UnixTime" trong DataFrame: df_log
        self.df_log["UnixTime"] = self.df_log.apply(self.convert_unixtime, axis=1)
                
        if self.keep_para:
            self.df_log["ParameterList"] = self.df_log.apply(
                self.get_parameter_list, 
                axis=1
            )
        self.df_log["IDLevel"] = self.df_log.apply(
            lambda row: dictionary_events.get(row["EventId"], {}).get(row["Level"], None),
            axis=1
        )
        self.df_log["LineId2"] = self.df_log.apply(
            lambda row: dictionary_events.get(row["EventId"], {}).get("TEMP", None),
            axis=1
        )
        self.df_log["BlockID"] = self.df_log["ParameterList"].apply(self.extract_block_id)
        
        print("Thực hiện lấy kết quả BlockID")
        result_list = self.extractByBlockID2()
        result_df = pd.DataFrame(list(result_list.items()), columns=['BlockID', 'Process'])
        result_df['Process'] = result_df['Process'].apply(lambda x: ', '.join(map(str, x)))
        
        # self.df_log.to_csv(
        #     os.path.join(self.savePath, self.logName + "_structured.csv"), index=False
        # )
        result_df.to_csv(
            os.path.join(self.savePath, self.logName.replace('.log', '') + "_process.csv"),
            index=False
        )
    
    def parse(self, logName):
        """
        Phương thức thực hiện quá trình phân tích log
        
        Tham số: 
        --------
            - `logName`: Tên file đầu vào thực hiện phân tích
        """
        print("File đầu vào: " + os.path.join(self.path, logName))
        start_time = datetime.now()
        self.logName = logName

        # ------Gán lại các đối tượng sử dụng--------#
        rootNode = self.rootNode       # Địa chỉ Node gốc
        logCluL = self.list_logClust   # Lưu giữ các đối tượng nhóm log Logcluster2 trước đó.
        templateLogCluL = []           # Lưu giữ các template đã có trước đó.

        # ----------------Load dữ liệu---------------#
        self.load_data() 

        #-----------------Phân tích------------------#
        # * Gán giá trị cho danh sách đếm template:
        unique_Level = self.df_log["Level"].unique()
        for value in unique_Level:
            if value not in self.list_counts:
                self.list_counts[value] = 0

        # * Thực hiện phân tích và lấy Template cho từng dòng thông điệp log
        count = 0
        for idx, line in self.df_log.iterrows(): 
            logLevel = str(line["Level"])
            logID = line["LineId"]
            logmessageL = self.preprocess(line["Content"]).strip().split()  
            matchCluster = self.treeSearch(rootNode, logmessageL)           
            if matchCluster is None:
                newCluster = Logcluster3(logTemplate=logmessageL, levelL=[logLevel], logIDL=[logID])
                
                # - Nếu có template mới, tạo ra, tăng biến đếm, gán id level template cho mẫu:
                self.list_counts["TEMP"] += 1
                self.list_counts[logLevel] += 1
                newCluster.idLevelTemp["TEMP"] = "L"+ str(self.list_counts["TEMP"])
                newCluster.idLevelTemp[logLevel] = logLevel[0] + str(self.list_counts[logLevel])
                
                # - Thêm chúng vào cây phân tích
                logCluL.append(newCluster)
                self.addSeqToPrefixTree(rootNode, newCluster)
            else:
                # - Nếu template đã có:
                newTemplate = self.getTemplate(logmessageL, matchCluster.logTemplate)
                matchCluster.logIDL.append(logID)
                # + Nếu có sự thay đổi trong template thành một template mới, cập nhật
                if " ".join(newTemplate) != " ".join(matchCluster.logTemplate):
                    matchCluster.logTemplate = newTemplate
                    
                # + Nếu có thêm một level mới biểu diễn template này, cập nhật nó
                if logLevel not in matchCluster.levelL:
                    matchCluster.addIDLevel(logLevel)
                    self.list_counts[logLevel] += 1
                    matchCluster.idLevelTemp[logLevel] = logLevel[0] + str(self.list_counts[logLevel])
                    
            count += 1
            if count % 100000 == 0 or count == len(self.df_log):
                print("Processed {0:.1f}% of log lines.".format(count * 100.0 / len(self.df_log))) # Chỉ sửa mỗi vị trí này.

        if not os.path.exists(self.savePath):
            os.makedirs(self.savePath)
        self.outputResult(self.list_logClust)
        print("Phân tích hoàn thành! [Thời gian thực hiện: {!s}]".format(datetime.now() - start_time))


In [37]:
input_dir  = '../../Dataset/HDFS/v1/'   # Đường dẫn vào thư mục chứa file đầu vào
output_dir = 'Result/'                  # Thư mục kết quả
log_file   = 'test2.log'                 # Tên file đầu vào
log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'  # HDFS log format
# Biểu thức chính quy
regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Ngưỡng tương đồng
depth      = 4  # Độ sâu của cây phân tích

parserObj = LogParser2(log_format, indir=input_dir, outdir=output_dir, depth=depth, st=st, rex=regex)
# parserObj.parse(log_file)

selected_columns = ['LineId', 'UnixTime', 'Level', 'EventId', 'EventTemplate', 'IDLevel', 'LineId2', 'ParameterList', 'BlockID']
# for i in range(1,6):
#     str_file = "test"+str(i)+".log"
#     parserObj.parse(str_file)
#     print(len(parserObj.list_logClust))
#     print(parserObj.list_counts)
#     # outputResult2(parserObj.list_logClust)
#     # parserObj.df_log.loc[:, selected_columns].head(60)
#     parserObj.df_log = pd.DataFrame()

parserObj.parse("HDFS_part1.log")
print(len(parserObj.list_logClust))
print(len(parserObj.df_log))
print(parserObj.list_counts)
# parserObj.df_log.loc[:, selected_columns].head(60)




File đầu vào: ../../Dataset/HDFS/v1/HDFS_part1.log
Total lines:  500000
Processed 20.0% of log lines.
Processed 40.0% of log lines.
Processed 60.0% of log lines.
Processed 80.0% of log lines.
Processed 100.0% of log lines.
Thực hiện lấy kết quả BlockID
Phân tích hoàn thành! [Thời gian thực hiện: 0:01:18.685433]
35
500000
{'TEMP': 35, 'INFO': 31, 'WARN': 4}


In [ ]:
# Đọc file gắn nhãn bất thường cho trước:
anomaly_df = pd.read_csv("HDFS.anomaly_label.csv")
anomaly_df.rename(columns={anomaly_df.columns[0]: "BlockID"}, inplace=True) # Đổi tên cột

# Đọc dữ liệu các DataFrame BLockID ghi lại tiến trình và cho vào một DataFrame:
file_list = glob.glob('./Result/*.csv')
df_list = [pd.read_csv(file) for file in file_list]
full_df = pd.concat(df_list, ignore_index=True)
process_df = full_df.groupby('BlockID')['Process'].apply(lambda x: ', '.join(x)).reset_index()
process_df['Process'] = process_df['Process'].apply(lambda x: x.split(', '))

print(len(process_df))

print(len(anomaly_df))

# Gộp 2 DataFrame thành 1 theo cột BLockId
merged_df_left = pd.merge(anomaly_df, process_df, on='BlockID', how='outer')
merged_df_left.to_csv(
            os.path.join("./Result","blockID_process.csv"),
            index=False
        )


In [ ]:
# # Đọc dữ liệu DataFrame từ nhiều file và gộp thành 1:
# file_list = glob.glob('./Test/*.csv')
# df_list = [pd.read_csv(file) for file in file_list]
# full_df = pd.concat(df_list, ignore_index=True)
# grouped_df = full_df.groupby('BlockID')['Process'].apply(lambda x: ', '.join(x)).reset_index()
# grouped_df['Process'] = grouped_df['Process'].apply(lambda x: x.split(', '))

# print(len(grouped_df))
# grouped_df.head(100)